# *Natural Language Processing Assignment*

##### <font color="teal">**1. Basic preprocessing**</font>






###### <font color="teal">**1.1 Open the database. Generate simple statistics about the abstracts. How many unique articles are there? What is the mean length of abstracts in characters?**</font>

In [1]:
import pandas as pd

data = pd.read_csv("PLOS_narrativity.csv")

unique_articles = data['pmid'].nunique()

mean_abstract_length = data['ab'].dropna().apply(len).mean()

print(f"Unique articles: {unique_articles}")
print(f"Mean length of abstracts: {mean_abstract_length:.2f} characters")


Unique articles: 802
Mean length of abstracts: 1496.18 characters


##### <font color="teal">**2. Word-level preprocessing**</font>


###### <font color="teal">**2.1 Split the abstracts into list of words. How many different words are there in the vocabulary?**</font>



In [3]:
all_abstracts = ' '.join(data['ab'].dropna())

words = set(all_abstracts.lower().split(sep=' '))

unique_word_count = len(words)

print(f"Unique words in abstracts: {unique_word_count}")


Unique words in abstracts: 19383


###### <font color="teal">**2.2 Split the abstracts into list of words using three different tokenizers from nltk. What is the difference in terms of number of words? What do you think has changed?**</font>



In [4]:
import nltk
from nltk.tokenize import TreebankWordTokenizer, ToktokTokenizer, TweetTokenizer

In [5]:
all_abstracts = ' '.join(data['ab'].dropna())

treebank_tokenizer = TreebankWordTokenizer()
toktok_tokenizer = ToktokTokenizer()
tweet_tokenizer = TweetTokenizer()

treebank_tokens = treebank_tokenizer.tokenize(all_abstracts)
toktok_tokens = toktok_tokenizer.tokenize(all_abstracts)
tweet_tokens = tweet_tokenizer.tokenize(all_abstracts)

treebank_unique_words = set(treebank_tokens)
toktok_unique_words = set(toktok_tokens)
tweet_unique_words = set(tweet_tokens)

print(f"Treebank Tokenizer: {len(treebank_unique_words)} unique words")
print(f"Toktok Tokenizer: {len(toktok_unique_words)} unique words")
print(f"Tweet Tokenizer: {len(tweet_unique_words)} unique words")


Treebank Tokenizer: 16576 unique words
Toktok Tokenizer: 16594 unique words
Tweet Tokenizer: 14432 unique words


Well, for the split word by space, we got 19383. 
Treebank got 16576, Toktok got 16594, and Tweet got 14432.

TreebankWordTokenizer:
Style: Based on the Penn Treebank tokenization.
Characteristics: Handles punctuation like periods and commas separately; splits contractions (e.g., "can't" -> "ca n't").

ToktokTokenizer:
Style: A general-purpose tokenizer.
Characteristics: Maintains a balance between splitting and grouping; respects some punctuation.

TweetTokenizer:
Style: Designed for social media text.
Characteristics: Keeps hashtags, mentions, and emoticons intact; better for informal language.

The different between the number of the words is depends on how they handle words and punctuation, contractions, and special symbols.
 


In [6]:
# print random sample of all tokenizer
import random

words_list = list(words)
treebank_unique_words_list = list(treebank_unique_words)
toktok_unique_words_list = list(toktok_unique_words)
tweet_unique_words_list = list(tweet_unique_words)

for i in range(20):
    # print(words_list[random.randint(0, len(words_list) - 1)])
    # print(treebank_unique_words_list[random.randint(0, len(treebank_unique_words_list) - 1)])
    print(toktok_unique_words_list[random.randint(0, len(toktok_unique_words_list) - 1)])
    # print(tweet_unique_words_list[random.randint(0, len(tweet_unique_words_list) - 1)])

pocket
samples
portions
specifics
7.8
impossible.
elect
Multidecadal
make
permitting
costs.
rates-canopy
requirements
locate
loggers.
2300
influenced
out
extrapolating
heliopora


##### <font color="teal">**3. Domain specificity and regex**</font>


###### <font color="teal">**3.1 Use regex to retrieve numbers (ints, floats, %, years, ...) in abstracts.**</font>


*Regex cheasheet* : see python's re module documentation https://docs.python.org/3/library/re.html  

*Other ressources* : 

- A good website to write and test regular expressions : 
https://regex101.com/
- A good game to learn regex : https://alf.nu/RegexGolf 


In [7]:
import re

pattern = r'\b\d+\.\d+|\b\d+%|\b\d{4}|\b\d+\b'
matches = re.findall(pattern, all_abstracts)
print(f"Found {len(matches)} numbers in the abstracts.")

Found 29883 numbers in the abstracts.


In [12]:
with open("test.txt", "w") as f:
    abstr = data['ab'].head(10)
    for i in range(len(abstr)):
        f.write(abstr[i])
        f.write("\n\n")

###### <font color="teal">**3.2 How many percent of characters are numbers (as defined above) in a given abstract?**</font>


In [10]:
characters = data['ab'].dropna().apply(len)

percentages = len(matches)/characters.sum() * 100
print(f"Found {percentages:.2f} percentages in the abstracts.")

Found 0.36 percentages in the abstracts.


##### <font color="teal">**4. Classic NLP pipeline**</font>


###### <font color="teal">**4.0 Re-tokenize using spacy**</font>

It is useful to take a look at spacy's [tokenizer documentation](https://spacy.io/usage/spacy-101#annotations-token)


In [2]:
print(data.columns)

Index(['Unnamed: 0', 'X_unit_id', 'X_created_at', 'X_id', 'X_started_at',
       'X_tainted', 'X_channel', 'X_trust', 'X_worker_id', 'X_country',
       'X_region', 'X_city', 'X_ip', 'appeal_to_reader', 'conjunctions',
       'connectivity', 'narrative_perspective', 'sensory_language', 'setting',
       'ab', 'appeal_to_reader_gold', 'conjunctions_gold', 'connectivity_gold',
       'narrative_perspective_gold', 'pmid', 'py', 'sensory_language_gold',
       'setting_gold', 'so', 'tc', 'af', 'au', 'bp', 'di', 'ep', 'is', 'pd',
       'pt', 'sn', 'ti', 'ut', 'vl', 'z9', 'cin_mas', 'firstauthor',
       'numberauthors', 'pid_mas', 'title'],
      dtype='object')


In [3]:
import spacy
nlp = spacy.load("en_core_web_sm") #load the small model
data["doc"] = data["ab"].apply(nlp)


In [4]:
data["tokens"] = data["doc"].apply(lambda doc: [token.text for token in doc])

###### <font color="teal">**4.1 Lemmatize using spacy**</font>


In [5]:
data["lemmas"] = data["doc"].apply(lambda doc: [token.lemma_ for token in doc])


In [6]:
print(data['lemmas'].head(5))

0    [the, range, of, plant, and, animal, be, move,...
1    [the, range, of, plant, and, animal, be, move,...
2    [the, range, of, plant, and, animal, be, move,...
3    [the, range, of, plant, and, animal, be, move,...
4    [the, range, of, plant, and, animal, be, move,...
Name: lemmas, dtype: object


###### <font color="teal">**4.2 POS tagging using spacy, plot the trees**</font>

In [7]:
data["pos_tags"] = data["doc"].apply(lambda doc: [(token.text, token.pos_) for token in doc])

In [8]:
from spacy import displacy

In [9]:
import IPython
print(IPython.__version__)
from IPython.display import *
print(dir())


8.0.0
['Audio', 'Code', 'DisplayHandle', 'DisplayObject', 'FileLink', 'FileLinks', 'GeoJSON', 'HTML', 'IFrame', 'IPython', 'Image', 'In', 'JSON', 'Javascript', 'Latex', 'Markdown', 'Math', 'Out', 'Pretty', 'ProgressBar', 'SVG', 'ScribdDocument', 'TextDisplayObject', 'Video', 'VimeoVideo', 'YouTubeVideo', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '__vsc_ipynb_file__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_i6', '_i7', '_i8', '_i9', '_ih', '_ii', '_iii', '_oh', 'clear_output', 'data', 'displacy', 'display', 'display_html', 'display_javascript', 'display_jpeg', 'display_json', 'display_latex', 'display_markdown', 'display_pdf', 'display_png', 'display_pretty', 'display_svg', 'exit', 'get_ipython', 'mean_abstract_length', 'nlp', 'pd', 'publish_display_data', 'quit', 'set_matplotlib_close', 'set_matplotlib_formats', 'spacy', 'unique_articles', 'update_display']


In [10]:
from IPython.display import display, HTML



In [11]:
for idx, row in data.iterrows():
    # Process the abstract text using spaCy
    doc = nlp(row["ab"])
    
    # Print which abstract is being processed
    print(f"Plotting tree for abstract {idx + 1}: {row['ab']}")
    
    # Render the dependency tree in Jupyter (using displacy.render)
    displacy.serve(doc, style="dep", options={"compact": True, "color": "blue", "background": "white"})

Plotting tree for abstract 1: The ranges of plants and animals are moving in response to recent changes in climate. As temperatures rise, ecosystems with 'nowhere to go', such as mountains, are considered to be more threatened. However, species survival may depend as much on keeping pace with moving climates as the climate's ultimate persistence. Here we present a new index of the velocity of temperature change (km yr(-1)), derived from spatial gradients ( degrees C km(-1)) and multimodel ensemble forecasts of rates of temperature increase ( degrees C yr(-1)) in the twenty-first century. This index represents the instantaneous local velocity along Earth's surface needed to maintain constant temperatures, and has a global mean of 0.42 km yr(-1) (A1B emission scenario). Owing to topographic effects, the velocity of temperature change is lowest in mountainous biomes such as tropical and subtropical coniferous forests (0.08 km yr(-1)), temperate coniferous forest, and montane grasslands. V

e:\School Project\NLP\NLP-Final\.venv\Lib\site-packages\spacy\displacy\__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
Plotting tree for abstract 2: The ranges of plants and animals are moving in response to recent changes in climate. As temperatures rise, ecosystems with 'nowhere to go', such as mountains, are considered to be more threatened. However, species survival may depend as much on keeping pace with moving climates as the climate's ultimate persistence. Here we present a new index of the velocity of temperature change (km yr(-1)), derived from spatial gradients ( degrees C km(-1)) and multimodel ensemble forecasts of rates of temperature increase ( degrees C yr(-1)) in the twenty-first century. This index represents the instantaneous local velocity along Earth's surface needed to maintain constant temperatures, and has a global mean of 0.42 km yr(-1) (A1B emission scenario). Owing to topographic effects, the velocity of temperature change is lowest in mountainous biomes such as tropical and subt


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



: 

: 

###### <font color="teal">**4.3 NER using spacy, give the amount of each entity type for a given abstract, visualize entities**</font>

In [11]:

# Assuming `data["doc"]` contains spaCy doc objects (after applying nlp)
data["ents"] = data["doc"].apply(lambda doc: [(ent.text, ent.label_) for ent in doc.ents])

# Print the first few rows to check
print(data["ents"].head())

0    [(the twenty-first century, DATE), (Earth, LOC...
1    [(the twenty-first century, DATE), (Earth, LOC...
2    [(the twenty-first century, DATE), (Earth, LOC...
3    [(the twenty-first century, DATE), (Earth, LOC...
4    [(the twenty-first century, DATE), (Earth, LOC...
Name: ents, dtype: object


In [12]:
from collections import Counter

# Example: Count entity types for one abstract
Counter([ent.label_ for ent in data.loc[0, "doc"].ents])

Counter({'DATE': 3, 'LOC': 2, 'QUANTITY': 3, 'PERCENT': 1})

In [13]:
displacy.render(data["doc"][0], style="ent")


##### <font color="teal">**5. Topic Modelling**</font>


###### <font color="teal">**5.1 Use Gensim's LDA to compute a topic model.**</font> 


In [46]:
import gensim
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.models import LdaModel
def preprocess_text(text):
    doc = nlp(text)
    tokens = [
        token.lemma_.lower()  # always lowercase the lemma
        for token in doc
        if token.lemma_.lower() not in STOP_WORDS and not token.is_punct and not token.is_space
    ]
    return tokens
# Tokenize + remove stopwords
data['clean_tokens'] = data['ab'].dropna().apply(preprocess_text)



In [47]:
print(data['clean_tokens'].head(10))

0    [range, plant, animal, response, recent, chang...
1    [range, plant, animal, response, recent, chang...
2    [range, plant, animal, response, recent, chang...
3    [range, plant, animal, response, recent, chang...
4    [range, plant, animal, response, recent, chang...
5    [range, plant, animal, response, recent, chang...
6    [range, plant, animal, response, recent, chang...
7    [polar, temperature, approximately, 3, 5, degr...
8    [polar, temperature, approximately, 3, 5, degr...
9    [polar, temperature, approximately, 3, 5, degr...
Name: clean_tokens, dtype: object


In [49]:
from gensim import corpora
dictionary = corpora.Dictionary(data['clean_tokens'])
corpus = [dictionary.doc2bow(text) for text in data['clean_tokens']]

In [50]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=15)

In [52]:
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.014*"coral" + 0.012*"increase" + 0.011*"temperature" + 0.011*"plant" + 0.010*"change"')
(1, '0.025*"specie" + 0.022*"change" + 0.020*"climate" + 0.011*"population" + 0.009*"model"')
(2, '0.016*"emission" + 0.014*"carbon" + 0.010*"use" + 0.010*"change" + 0.009*"climate"')
(3, '0.017*"forest" + 0.016*"change" + 0.015*"climate" + 0.013*"increase" + 0.012*"tree"')
(4, '0.017*"temperature" + 0.017*"change" + 0.016*"climate" + 0.011*"ocean" + 0.010*"ice"')


###### <font color="teal">**5.2 Use PyLDAvis to visualise the topic model. What are the different topic clusters?**</font>


In [53]:
import pyLDAvis.gensim_models
import pyLDAvis

# Prepare visualization
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.056157  0.081542       1        1  27.430994
2     -0.223074  0.037988       2        1  19.203435
4      0.023344 -0.162290       3        1  18.748561
0      0.100774  0.100964       4        1  18.203110
3      0.042798 -0.058205       5        1  16.413901, topic_info=            Term         Freq        Total Category  logprob  loglift
85        specie  6971.000000  6971.000000  Default  30.0000  30.0000
26      emission  2363.000000  2363.000000  Default  29.0000  29.0000
33        forest  2650.000000  2650.000000  Default  28.0000  28.0000
782        coral  1943.000000  1943.000000  Default  27.0000  27.0000
93   temperature  5383.000000  5383.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
406        cause   385.415028  1250.454211   Topic5  -5.7818   0.6301
191    condition   404.887428  1971.380303   Topic5  -5.7325   0.2242
89       suggest   406.139493  2249.659692   Topic5  -5.7295   0.0952
267       impact   405.146096  2751.011871   Topic5  -5.7319  -0.1084
314       future   396.044326  2265.529846   Topic5  -5.7546   0.0630

[419 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
1181      5  0.983481              2.7
4228      4  0.993790               a.
1132      4  0.982560  acclimatization
610       1  0.954058       adaptation
610       2  0.002234       adaptation
...     ...       ...              ...
888       2  0.715267            yield
888       3  0.060179            yield
888       4  0.147868            yield
2389      1  0.010838          younger
2389      3  0.986229          younger

[868 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 5, 1, 4])

###### <font color="teal">**5.3 Use a tf-idf representation for each abstract, and use your favorite clustering algorithm.**</font>


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
data['clean_text'] = data['clean_tokens'].apply(lambda tokens: ' '.join(tokens))



In [58]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)  # You can tune these
tfidf_matrix = vectorizer.fit_transform(data['clean_text'].dropna())

In [ ]:
from sklearn.cluster import KMeans

# Define number of clusters
num_clusters = 813
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# Add cluster labels to the dataframe
data.loc[data['clean_text'].notnull(), 'cluster'] = kmeans.labels_

e:\School Project\NLP\NLP-Final\.venv\Lib\site-packages\sklearn\base.py:1473: ConvergenceWarning: Number of distinct clusters (802) found smaller than n_clusters (813). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [92]:
for i in range(num_clusters):
    print(f"\nCluster {i} Examples:")
    print(data[data['cluster'] == i]['ab'].head(3))


Cluster 0 Examples:
2100    Climate change, rising atmospheric carbon diox...
2101    Climate change, rising atmospheric carbon diox...
2102    Climate change, rising atmospheric carbon diox...
Name: ab, dtype: object

Cluster 1 Examples:
5334    This study explores potential adaptation appro...
5335    This study explores potential adaptation appro...
5336    This study explores potential adaptation appro...
Name: ab, dtype: object

Cluster 2 Examples:
4676    Understanding how communities respond to chang...
4677    Understanding how communities respond to chang...
4678    Understanding how communities respond to chang...
Name: ab, dtype: object

Cluster 3 Examples:
2940    Forests and their soils contain the majority o...
2941    Forests and their soils contain the majority o...
2942    Forests and their soils contain the majority o...
Name: ab, dtype: object

Cluster 4 Examples:
364    Climate change is predicted to be one of the g...
365    Climate change is predicted to be one o

KeyboardInterrupt: 

In [93]:
from sklearn.metrics import silhouette_score

score = silhouette_score(tfidf_matrix, kmeans.labels_)
print(f"Silhouette Score: {score:.3f}")


Silhouette Score: 1.000
